In [1]:
import mujoco
import mujoco_viewer
import random
import numpy as np
import itertools
import pickle
import time

SENSOR_NUM = 4
SENSOR_MAX = 0.18
SENSOR_DIV = 4
ANGLE_DIV = 16
ACTION_NUM = 7
STATE_NUM = (SENSOR_DIV**SENSOR_NUM)*ANGLE_DIV
ACT_MAX = 0.7


LEARN_INTEVAL = 0.1


#--- この強化学習の目的は ---
#マイクロマウスが止まることなく前進し続ける
#前進できていればプラス
#後進したらマイナス

#---- Stateをどう設定するか？ ---
#Sensorの状態
#マウスの位置の推定値
#マウスの速度の推定値
#マウスの方向の推定値

#--- Rewardはどう計算するのか ---

def sensor2state(lf, ls, rs, rf, sh):
    lf = int(lf>sh)
    ls = int(ls>sh)
    rs = int(rs>sh)
    rf = int(rf>sh)
    #print(lf,ls,rs,rf)
    return lf*8 + ls*4+ rs*2 + rf


class q_learn:
    def __init__(self, num_state, num_action):
        self.qtable=np.zeros(shape=(num_state, num_action))
        self.filepath='./qtable02'
    
    
    def update(self, state, action, reward, next_state):
        gamma = 0.99
        alpha = 0.5
        #print(next_state)
        next_Max_Q=max(self.qtable[next_state])
        self.qtable[state, action] = (1 - alpha) * self.qtable[state, action] +\
                alpha * (reward + gamma * next_Max_Q)
        
    def get_action(self, next_state):
        #徐々に最適行動のみをとる、ε-greedy法
        #epsilon = 0.5 * (1 / (episode + 1))
        epsilon = 0.1
        if epsilon <= np.random.uniform(0, 1):
            next_action = np.argmax(self.qtable[next_state])
        else:
            next_action = random.randint(0,ACTION_NUM-1)#---------------------------変更する
        return next_action


    def save_weights(self, filepath=None):
        """ 方策のパラメータの保存 """
        # Qテーブルの保存
        if filepath is None:
            filepath = self.filepath + '.pkl'
        with open(filepath, mode='wb') as f:
            pickle.dump(self.qtable, f)

    def load_weights(self, filepath=None):
        """ 方策のパラメータの読み込み """
        
        # Qテーブルの読み込み
        if filepath is None:
            filepath = self.filepath + '.pkl'
        with open(filepath, mode='rb') as f:
            self.qtable = pickle.load(f) 
        

class env:

    def __init__(self):

        #行動リストを作る
        self.act_list=[[1,1],[1,0.5],[0.5,1],[0.5,0,5],[0,0],[-0.5,-0.5],[0.1,-0.1]]
        #act=list(np.linspace(-1, 1, ACTION_NUM))
        #self.act_list=list(itertools.product(act, repeat=2))
        #print(self.act_list)

        #self.act=[[1,1],[1,0],[0,1],[0,0],[0,-1],[-1,0],[-1,-1],[1,-1],[1,-1],[-1,1]]
        #self.wall_pattern=[7,8,6,8,3,8,2,8,8,5,8,4,8,1,8,0]
        #self.rwd_wall = [[ 2, 2, 2,-1,-1,-1,-1, 0, 0],\
        #            [ 2,-1, 2,-1,-1,-1,-1, 0, 0],\
        #            [-1, 2,	2,-1,-1,-1,-1, 0, 0],\
        #            [-1,-1,	2,-1,-1,-1,-1, 0, 0],\
        #            [ 2, 2,-1,-1,-1,-1,-1, 0, 0],\
        #             [ 2,-1,-1,-1,-1,-1,-1, 0, 0],\
        #            [-1, 2,-1,-1,-1,-1,-1, 0, 0],\
        #            [-1,-1,-1, 0,-1,-1,-1, 2, 2],\
        #            [ 0, 0, 0, 0, 0, 0, 0, 1, 0]]

        self.model = mujoco.MjModel.from_xml_path('./mouse_in_maze.xml')#modelを読み込んで用意 インスタンスに用意しておく
        self.data = mujoco.MjData(self.model) #モデルのデータを用意
        self.viewer = mujoco_viewer.MujocoViewer(self.model,self.data) #mujoco描画起動
        self.init_pos = self.data.qpos #ロボットの初期位置座標取得
        self.init_vel = self.data.qvel #ロボットの初期速度取得
        stepcount = 0
        self.wheel_left_id = mujoco.mj_name2id(self.model, 3,'left wheel joint')
        self.wheel_right_id = mujoco.mj_name2id(self.model, 3,'right wheel joint')
        self.wheel_ang_left = self.data.qpos[7]
        self.wheel_ang_right = self.data.qpos[8]
        self.actuator_right_id = mujoco.mj_name2id(self.model, mujoco.mjtObj.mjOBJ_ACTUATOR, 'right')
        self.actuator_left_id  = mujoco.mj_name2id(self.model, mujoco.mjtObj.mjOBJ_ACTUATOR, 'left')
        self.lf_id = mujoco.mj_name2id(self.model, mujoco.mjtObj.mjOBJ_SENSOR, 'LF')
        self.ls_id = mujoco.mj_name2id(self.model, mujoco.mjtObj.mjOBJ_SENSOR, 'LS')
        self.rs_id = mujoco.mj_name2id(self.model, mujoco.mjtObj.mjOBJ_SENSOR, 'RS')
        self.rf_id = mujoco.mj_name2id(self.model, mujoco.mjtObj.mjOBJ_SENSOR, 'RF')
        self.vel_y_id = 1 
        self.angle_id = 3
    
    def reset(self):
        mujoco.mj_resetData(self.model, self.data)
        mujoco.mj_forward(self.model, self.data)

    def step(self):
        info = 0
        done = self.check_conflict()
        state = self.get_state()
        reward = self.reward() 
        return state, reward, done, info

    def get_distance(self):
        lf = self.data.sensordata[self.lf_id]
        ls = self.data.sensordata[self.ls_id]
        rs = self.data.sensordata[self.rs_id]
        rf = self.data.sensordata[self.rf_id]
        return lf,ls,rs,rf

    def distance_digitize(self):
        delta = SENSOR_MAX/SENSOR_DIV
        lf,ls,rs,rf = self.get_distance()
        lf = int(lf/delta)
        ls = int(ls/delta)
        rs = int(rs/delta)
        rf = int(rf/delta)
        return lf,ls,rs,rf

    def get_angle(self):
        return 360*(self.data.qpos[self.angle_id]+1.0)/2

    def angle_digitize(self):
        delta = 360/ANGLE_DIV
        angle = self.get_angle()
        return int(angle/delta)

    def get_velocity(self):
        return self.data.qvel[self.vel_y_id]

    def action(self, act):
        self.data.ctrl[self.actuator_right_id]= self.act_list[act][0] * ACT_MAX
        self.data.ctrl[self.actuator_left_id] = self.act_list[act][1] * ACT_MAX

    def get_state(self):
        lf, ls, rs, rf = self.distance_digitize()
        angle = self.angle_digitize()
        #print(lf,ls,rs,rf,angle,lf*2**10 + ls*2**8 + rs*2**6 + rf*2*4 + angle)
        if lf>3 or ls>3 or rs>3 or rf>3:
            print("Range digitize faluer",lf,ls,rs,rf)
        if angle>16:
            print("angle digitize failuer", angle)

        return lf*2**10 + ls*2**8 + rs*2**6 + rf*2**4 + angle

    def reward(self):
        #前進できていたら報酬、後退または停止したら罰
        velocity = self.get_velocity()
        if velocity>0:
            rwd = 2
        else:
            rwd = -3

        if self.check_conflict():
            rwd = rwd - 10

        return rwd
    
    def check_conflict(self):
        hit_wall_f = self.data.sensordata[4].copy() #フォースセンサの値取得
        hit_wall_b = self.data.sensordata[5].copy() #フォースセンサの値取得
        #print(self.data.sensordata)
        #print(hit_wall_f, hit_wall_b)
        if (hit_wall_b>0 or hit_wall_f>0):
            return True
        else:
            return False 
#-------------------------------------------------------------------------------------------

mouse =env()
Q_learn = q_learn(STATE_NUM, ACTION_NUM)  

#Q_learn.load_weights()
# simulate and render
oldtime=mouse.data.time
rendertime=mouse.data.time
action = random.randint(0, ACTION_NUM-1)
state = mouse.get_state()
itacount = 0
olditacount=0
rewardsum = 0
Ita=[]
Reward=[]

# simulate and render
for _ in range(10000):
    if mouse.viewer.is_alive:
        mujoco.mj_step(mouse.model, mouse.data)
        mouse.viewer.render()
    else:
        break

# close
mouse.viewer.close()




Pressed ESC
Quitting.
